In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('ProjectExcercise_dataframe').getOrCreate()

In [0]:
df=spark.read.table('walmart_stock')
df.head(5)

Out[3]: [Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [0]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Adj Close: double (nullable = true)



In [0]:
df.columns

Out[5]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [0]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [0]:
df_modified=df.withColumn('HV Ratio',df['High']/df['Volume'])

In [0]:
df_modified.columns

Out[8]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'HV Ratio']

In [0]:
df_modified.select(df_modified['HV Ratio']).show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [0]:
df_modified=df_modified.orderBy(df_modified['High'].desc()).collect()

In [0]:
df_modified[0][0]

Out[11]: datetime.date(2015, 1, 13)

In [0]:
df_modified['Date']

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-1839103271906903> in <cell line: 1>()
----> 1 df_modified['Date']

TypeError: list indices must be integers or slices, not str

In [0]:
df.select('Close').show


Out[31]: <bound method DataFrame.show of DataFrame[Close: double]>

In [0]:
from pyspark.sql.functions import mean,count,length,corr,year
df.agg({'Close': 'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [0]:
df.agg({'Volume':'max'}).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [0]:
df.agg({'Volume':'min'}).show()

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



In [0]:
frame=df.filter('Close<60').select('Close','Date')

In [0]:
frame

Out[46]: DataFrame[Close: double, Date: date]

In [0]:
frame.select(count('Date')).show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



In [0]:
day=df.filter("High>80").select(count("High")).collect()

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<command-2555398597203645> in <cell line: 1>()
----> 1 day[0][1]

/databricks/spark/python/pyspark/sql/types.py in __getitem__(self, item)
   1952     def __getitem__(self, item: Any) -> Any:
   1953         if isinstance(item, (int, slice)):
-> 1954             return super(Row, self).__getitem__(item)
   1955         try:
   1956             # it will be slow when it has many fields,

IndexError: tuple index out of range

In [0]:
total_day=df.select("Date").select(count("Date")).collect()

In [0]:
day[0][0]/total_day[0][0]*100

Out[83]: 9.141494435612083

In [0]:
#finding correlation
df.select(['High','Volume']).select(corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [0]:
df.select([year("Date").alias("Year"),"High"]).groupBy("Year").max().show()

+----+---------+---------+
|Year|max(Year)|max(High)|
+----+---------+---------+
|2015|     2015|90.970001|
|2013|     2013|81.370003|
|2014|     2014|88.089996|
|2012|     2012|77.599998|
|2016|     2016|75.190002|
+----+---------+---------+

